# 创建实时推理服务

在训练了预测模型之后，可以将其部署为实时服务，客户端可以使用该服务从新数据中获取预测。

## 连接到工作区

首先，请连接到你的工作区。

> **备注**：如果尚未与 Azure 订阅建立经过身份验证的会话，则系统将提示你通过执行以下操作进行身份验证：单击链接，输入验证码，然后登录到 Azure。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 从保存的配置文件加载工作区
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 训练和注册模型

现在我们来训练并注册模型。

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 在工作区中创建 Azure ML 试验
experiment = Experiment(workspace=ws, name="mslearn-train-diabetes")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# 加载糖尿病数据集
print("Loading Data...")
diabetes = pd.read_csv('data/diabetes.csv')

# 分隔特征和标签
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 将数据拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 训练决策树模型
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 计算准确度
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# 计算 AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# 保存训练的模型
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# 完成运行
run.complete()

# 注册模型
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

## 将模型部署为 Web 服务

你已经训练并注册了一个机器学习模型，该模型可以根据患者患上糖尿病的可能性对他们进行分类。此模型可在生产环境中使用，例如医生的手术室（在此场景中，只有被认为有风险的患者需要进行糖尿病临床测试）。为了支持此场景，你需要将模型部署为 Web 服务。

首先，让我们确定你在工作区中注册了哪些模型。

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

现在我们来获取要部署的模型。默认情况下，如果我们指定模型名称，将返回最新版本。

In [ ]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

我们将创建一个 Web 服务来托管此模型，这将需要一些代码和配置文件；因此，我们先为它们创建一个文件夹。

In [ ]:
import os

folder_name = 'diabetes_service'

# 为 Web 服务文件创建文件夹
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# 设置评分脚本的路径
script_file = os.path.join(experiment_folder,"score_diabetes.py")

我们在其中部署模型的 Web 服务将需要一些 Python 代码来加载输入数据、从工作区获取模型以及生成和返回预测。我们将此代码保存在将部署到 Web 服务的 *入口脚本* （通常称为*评分脚本*）中：

In [ ]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# 加载服务时调用
def init():
    global model
    # 获取已部署模型文件的路径并加载它
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)

# 收到请求时调用
def run(raw_data):
    # 获取输入数据，作为 numpy 数组
    data = np.array(json.loads(raw_data)['data'])
    # 从模型中获取预测
    predictions = model.predict(data)
    # 获取每个预测对应的类名（0 或 1）
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # 返回 JSON 形式的预测
    return json.dumps(predicted_classes)

该 Web 服务将托管在容器中，该容器在进行初始化时需要安装任何所需的 Python 依赖项。在本例中，我们的评分代码需要 **scikit-learn**，因此我们将创建一个 .yml 文件，该文件会指示容器主机将其安装到环境中。

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

# 添加模型的依赖项（AzureML 默认值已包含在内）
myenv = CondaDependencies()
myenv.add_conda_package('scikit-learn')

# 将环境配置保存为 .yml 文件
env_file = os.path.join(experiment_folder,"diabetes_env.yml")
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# 打印 .yml 文件
with open(env_file,"r") as f:
    print(f.read())

现在你可以进行部署了。我们将为容器部署名为 **diabetes-service** 的服务。部署过程包括以下步骤：

1.定义一个推理配置，其中包括加载和使用模型所需的评分和环境文件。
2.定义一个部署配置，该配置定义将托管此服务的执行环境。在本例中为 Azure 容器实例。
3.将模型部署为 Web 服务。
4.验证已部署服务的状态。

> **更多信息**： 有关模型部署的更多详细信息以及目标执行环境选项，请参阅此[文档](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where)。

部署会花费一些时间，因为它首先运行一个进程以创建容器映像，然后会运行一个进程以基于该映像创建 Web 服务。成功完成部署后，你会看到  **“正常”** 状态。

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# 配置评分环境
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "diabetes-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

希望部署已成功，然后你就能看到 **“正常”** 状态。如果未成功，可以使用以下代码来获取服务日志以帮助你解决问题。

In [ ]:
print(service.get_logs())

# 如果需要进行更改和重新部署，则可能需要使用以下代码删除运行不正常的服务：
#service.delete()

在 [Azure 机器学习工作室](https://ml.azure.com)中查看工作区，然后查看 **“终结点”** 页面，此页面显示工作区中已部署的服务。

还可以通过运行以下代码来检索工作区中 Web 服务的名称：

In [ ]:
for webservice_name in ws.webservices:
    print(webservice_name)

## 使用 Web 服务

部署此服务后，现在可以从客户端应用程序使用它。

In [ ]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# 将该数组转换为 JSON 文档中的可序列化列表
input_json = json.dumps({"data": x_new})

# 调用 Web 服务，传递输入数据（该 Web 服务还会接受二进制格式的数据）
predictions = service.run(input_data = input_json)

# 获取预测的类 - 它将是第一个（也是唯一一个）类。
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

还可以将多位患者的观察结果发送到此服务，并获取针对每位患者的预测。

In [ ]:
import json

# 这次我们的输入是由两个特征数组组成的数组
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# 将该数组或多个数组转换为 JSON 文档中的可序列化列表
input_json = json.dumps({"data": x_new})

# 调用 Web 服务，以传递输入数据
predictions = service.run(input_data = input_json)

# 获取预测的类。
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

上述代码使用 Azure 机器学习 SDK 连接到容器化 Web 服务，并将其用于根据糖尿病分类模型生成预测。在生产环境中，不使用 Azure 机器学习 SDK 而是仅向 Web 服务发出 HTTP 请求的业务应用程序可能使用模型。

我们来确定这些应用程序必须将其请求提交到的 URL：

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

你已知道终结点 URI，应用程序现在可以发出 HTTP 请求、发送 JSON 格式的患者数据以及接收预测的类。

In [ ]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# 将该数组转换为 JSON 文档中的可序列化列表
input_json = json.dumps({"data": x_new})

# 设置内容类型
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

你已将 Web 服务部署为不需要进行身份验证的 Azure 容器实例 (ACI) 服务。这对于开发和测试是可行的，但是对于生产，应考虑部署到 Azure Kubernetes 服务 (AKS) 群集并启用基于令牌的身份验证。这要求 REST 请求包含一个**授权**标头。

## 删除服务

如果你不再需要服务，应将其删除以免产生不必要的费用。

In [ ]:
service.delete()
print ('Service deleted.')

有关将模型作为服务发布的详细信息，请参阅此[文档](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where)